In [53]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
%%file GRBL.py
import serial
class GRBL(object):
    BAUDRATE = 115200
    
    def __init__(self, port):
        self.serial = serial.Serial(port=port,
                                    baudrate=GRBL.BAUDRATE,
                                    timeout=0.10)
        
    def write(self, command_line=""):
        self.serial.flushInput()
        self.serial.write("\n".encode())
        self.serial.write("{cmd}\n".format(cmd=command_line).encode())
        
    def read(self, multiline=True):
        if multiline:
            responses = self.serial.readlines()
            responses = [response.decode().strip() for response in responses]
            return responses
        else:
            response = self.serial.readline()
            return response.decode().strip()
 
    def cmd(self, command_line, resp=True, multiline=True):
        self.write(command_line)
        if resp:
            return self.read(multiline=multiline)
        return None
        

settings = [
    ("$0", "step_pulse"),
    ("$1", "step_idle_delay"),
    ("$2", "step_port_invert"),
    ("$3", "direction_port_invert"),
    ("$4", "step_enable_invert"),
    ("$5", "limit_pin_invert"),
    ("$6", "probe_pin_invert"),
    ("$10", "status_report"),
    ("$11", "junction_deviation"),
    ("$12", "arc_tolerance"),
    ("$13", "report_inches"),
    ("$20", "soft_limits"),
    ("$21", "hard_limits"),
    ("$22", "homing_cycle"),
    ("$23", "homing_dir_invert"),
    ("$24", "homing_feed"),
    ("$25", "homing_seek"),
    ("$26", "homing_debounce"),
    ("$27", "homing_pull_off"),
    ("$30", "max_spindle_speed"),
    ("$31", "min_spindle_speed"),
    ("$32", "laser_mode"),
    ("$100", "x_steps_mm"),
    ("$101", "y_steps_mm"),
    ("$102", "z_steps_mm"),
    ("$110", "x_max_rate"),
    ("$111", "y_max_rate"),
    ("$112", "z_max_rate"),
    ("$120", "x_acceleration"),
    ("$121", "y_acceleration"),
    ("$122", "z_acceleration"),
    ("$130", "x_travel"),
    ("$131", "y_travel"),
    ("$132", "z_travel"),
    ]

def grbl_getter_generator(cmd):
    def grbl_getter(self):
        config = self.cmd("$$", resp=True, multiline=True)
        for config_line in config:
            if config_line.startswith("$"):
                key, value = config_line.split("=")
                if key == cmd:
                    return float(value)
        return None
    return grbl_getter
    
def grbl_setter_generator(cmd):
    def grbl_setter(self, value):
        set_cmd = "{cmd}={value}".format(cmd=cmd, value=value)
        ret = self.cmd(set_cmd, resp=True, multiline=False)
        print(ret)
        
    return grbl_setter

for setting in settings:
    cmd = setting[0]
    name = setting[1]
    
    setter = grbl_setter_generator(cmd)
    getter = grbl_getter_generator(cmd)
    
    prop = property(fget=getter,
                    fset=setter,
                    doc=" ".join(name.split("_")))
    
    setattr(GRBL, name, prop)

Overwriting GRBL.py


In [63]:
%aimport GRBL

In [64]:
grbl = GRBL.GRBL("/dev/cnc_3018")

# Grbl '$' Commands

In [65]:
grbl.cmd("?")

['<Hold:0|MPos:0.000,0.000,0.000|FS:0,0>']

In [66]:
grbl.cmd("$?")

['<Hold:0|MPos:0.000,0.000,0.000|FS:0,0>']

In [77]:
grbl.cmd("$#") # View gcode parameters

['ok',
 '[G54:0.000,0.000,0.000]',
 '[G55:0.000,0.000,0.000]',
 '[G56:0.000,0.000,0.000]',
 '[G57:0.000,0.000,0.000]',
 '[G58:0.000,0.000,0.000]',
 '[G59:0.000,0.000,0.000]',
 '[G28:0.000,0.000,0.000]',
 '[G30:0.000,0.000,0.000]',
 '[G92:0.000,0.000,0.000]',
 '[TLO:0.000]',
 '[PRB:0.000,0.000,0.000:0]',
 'ok']

In [80]:
grbl.cmd("$G")

['ok', '[GC:G0 G54 G17 G21 G90 G94 M5 M9 T0 F0 S0]', 'ok']

In [81]:
grbl.cmd("$I")

['ok', '[VER:1.1f.20170801:]', '[OPT:V,15,128]', 'ok']

In [82]:
grbl.cmd("$N")

['ok', '$N0=', '$N1=', 'ok']

In [83]:
grbl.cmd("$C")

['ok', '[MSG:Enabled]', 'ok']

In [84]:
grbl.cmd("$C")

['ok', '[MSG:Disabled]', 'ok', '', "Grbl 1.1f ['$' for help]"]

In [34]:
grbl.cmd("$X")

['ok', 'ok']

In [ ]:
# Not until I'm on the machine.
#grbl.cmd("$H")

ASCII Realtime Command Descriptions

Four realtime commands are type-able by users on a keyboard and shown in the $ Grbl help message. These realtime command characters control some of Grbl's basic functions.

In [76]:
grbl.cmd("\x18")

['', "Grbl 1.1f ['$' for help]", 'ok']

In [75]:
grbl.cmd("?")

['<Hold:0|MPos:0.000,0.000,0.000|FS:0,0>']

In [72]:
grbl.cmd("~")

['ok', 'ok']

In [73]:
grbl.cmd("!")

['ok']

In [40]:
ser = grbl.serial

In [46]:
grbl.write("$$")
exit = 0
ser.timeout = 0.001
msg = b''
while exit < 2:
    more = ser.read(200)
    print(more)
    msg += more
    if len(more) == 0:
        exit += 1
    else:
        exit = 0

b''
b''


In [49]:
ser.timeout=0.05

In [51]:
grbl.write("$$")
ser.readlines()

[]